In [ ]:
# Enable autoreload to automatically reload modules when they are updated
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
from typing import Dict, Any, Optional, Union
import joblib
import psutil, shutil
import os
import numpy as np
from sarpyx.processor.core.focus import CoarseRDA
from sarpyx.utils.zarr_utils import ZarrManager, dask_slice_saver, concatenate_slices_efficient

# Only add path if not already added
cwd = Path.cwd()
sarpyx_path = cwd.parent
if str(sarpyx_path) not in __import__('sys').path:
    __import__('sys').path.append(str(sarpyx_path))
    
# Output directory for focused data
output_dir = cwd.parent / 'focused_data' 
tmp_dir = output_dir / 'tmp'

In [ ]:
# ------- Input -------
file_path = "/Data_large/marine/PythonProjects/SAR/sarpyx/decoded_data/s1a-s1-raw-s-hh-20150519t121652-20150519t121725-005990-007b7d_burst_0.zarr"
# ------- Configure the handler for processing -------
productName = Path(file_path).stem
N_SLICES = 5
# ------- Init -------
# STEP1: Data Focusing Preparation
handler = ZarrManager(file_path)
# Starting focusing in steps:
for SLICE_N in range(N_SLICES): 
    raw_data = handler.get_slice_block(slice_idx=SLICE_N, N_blocks=N_SLICES)
    print(f'📊 Sliced raw data shape: {raw_data["echo"].shape} ')

    processor = CoarseRDA(
                raw_data=raw_data,  # type: ignore
                verbose=False,
            )
    print(f'🛠️ Processor initialized with raw data of shape: {raw_data["echo"].shape}')
    processor.data_focus()
    
    raw = processor.raw_data
    rc = processor.range_compressed_data
    rcmc = processor.rcmc_data
    az = processor.azimuth_compressed_data
    metadata = raw_data['metadata'].to_dict()
    ephemeris = raw_data['ephemeris'].to_dict()
    
    # Ensure all data types are as expected
    assert isinstance(metadata, dict), f'Expected metadata to be dict, got {type(metadata)}'
    assert isinstance(ephemeris, dict), f'Expected ephemeris to be dict, got {type(ephemeris)}'
    assert isinstance(raw, np.ndarray), f'Expected raw data to be ndarray, got {type(raw)}'
    assert isinstance(rc, np.ndarray), f'Expected range compressed data to be ndarray, got {type(rc)}'
    assert isinstance(rcmc, np.ndarray), f'Expected rcmc data to be ndarray, got {type(rcmc)}'
    assert isinstance(az, np.ndarray), f'Expected azimuth compressed data to be ndarray, got {type(az)}'

    result = {'raw': raw, 'rc': rc, 'rcmc': rcmc, 'az': az, 'metadata': metadata, 'ephemeris': ephemeris}
    zarr_path = tmp_dir / f"processor_slice_{SLICE_N}.zarr"
    dask_slice_saver(result, zarr_path, chunks='auto', clevel=7)
    
    del processor, raw_data, raw, rc, rcmc, az, metadata, ephemeris, result  # Clear memory
    
    print(f'💾 Slice {SLICE_N+1} saved to: {tmp_dir / f"processor_slice_{SLICE_N+1}.pkl"}')
 
# --------- Concatenate slices efficiently ---------
# STEP2: Cleanup: delete temporary dir
tmp_files = [tmp_dir / f'processor_slice_{i}.zarr' for i in range(N_SLICES)]
print(f'🔗 Concatenating {len(tmp_files)} slices...')
concatenated_data = concatenate_slices_efficient(tmp_files, output_dir / f'{productName}.zarr') # type: ignore
print(f'✅ Concatenated data saved to: {output_dir / f"{productName}.zarr"}')
# --------- CLEANUP ---------
if tmp_dir.exists():
    for item in tmp_dir.iterdir():
        if item.is_file():
            item.unlink()
        elif item.is_dir():
            shutil.rmtree(item)
    tmp_dir.rmdir()
    print(f'🗑️ Temporary directory {tmp_dir} cleaned up.')